In [1]:
import json

In [2]:
class Board:

    # Class Variable
    type = '9x9'

    box_map = {(0, 0) : 0,
    (0, 1) : 1,
    (0, 2) : 2,
    (1, 0) : 3,
    (1, 1) : 4,
    (1, 2) : 5,
    (2, 0) : 6,
    (2, 1) : 7,
    (2, 2) : 8 }

    # The init method or constructor
    def __init__(self, chunk):
        lines = chunk.split('\n')
        # Instance Variables
        self.orig_board = chunk
        self.name = lines[0]
        self.rows = list(map(list, lines[1:10]))      
        self.empty_squares = {}
        self.solve_log = []
        
        self._calc_empties()
        self.update_candidate_notes()

    def __str__(self):
        st = f"{self.name}\n{self.rows}"
        return st.replace('],', '],\n')

    def _calc_empties(self):
        for r in range(0,9):
            row = self.get_row(r)
            for c in range(0,9):
                if row[c] == '0':
                    self.empty_squares[(r,c)] = None

    def calc_candidates(self, square):
        (r,c) = square
        cands = set(list('123456789'))
#        print(f'row {r}: {self.get_row(r)}')
#        print(f'col {c}: {self.get_col(c)}')
        b = self.get_box_for_square(square)
#        print(f'box {b}: {self.get_box(b)}')
        cands = cands.difference(self.get_row(r))
        cands = cands.difference(self.get_col(c))
        cands = cands.difference(self.get_box(self.get_box_for_square(square)))
        self.empty_squares[square] = cands

    def set_square(self, square, val):
        (r, c) = square
        self.get_row(r)[c] = val

    def get_row(self, n):
        return self.rows[n]

    def get_col(self, n):
        return list(map(lambda x: x[n], self.rows))

    def get_box(self, n, flat_list=True):
        b_row = 3 * (n // 3)
        b_col = 3 * (n % 3)
        box_matrix = [self.rows[b_row  ][b_col:b_col+3], 
                      self.rows[b_row+1][b_col:b_col+3], 
                      self.rows[b_row+2][b_col:b_col+3] ]
        if flat_list:
            return flatten(box_matrix)
        else:
            return box_matrix     
            
    def get_rows(self):
        return self.rows

    def get_cols(self):
        return list(map(lambda x: self.get_col(x), range(0,9)))
        
    def get_boxes(self):
        return list(map(lambda x: self.get_box(x), range(0,9)))
        
    def get_box_row(self, n):
        if n==0:
            return b.rows[0:3]
        if n==1:
            return b.rows[3:6]
        if n==2:
            return b.rows[6:9]
            
        return 'No way Jose'
        
    def get_box_col(self, n):
        return [b.get_col(0), b.get_col(1), b.get_col(2)]

    def get_box_for_square(self, square):
        # return self.get_box(0)
        (r, c) = square
        b_row = r//3
        b_col = c//3
        # print(f'box: {(b_row, b_col)}')
        return Board.box_map[(b_row, b_col)]

    def get_rows_containing_digit(self, digit):
        return [i for i, x in enumerate(map(lambda x: digit in x, b.get_rows())) if x]

    def get_cols_containing_digit(self, digit):
        return [i for i, x in enumerate(map(lambda x: digit in x, b.get_cols())) if x]

    def get_boxes_containing_digit(self, digit):
        return [i for i, x in enumerate(map(lambda x: digit in x, b.get_boxes())) if x]

    def update_candidate_notes(self):
        for sq in self.empty_squares:
            self.calc_candidates(sq)
        self.solve_log.append(f'{len(self.empty_squares)} squares open. Candidates per square summary: {self.summarize_board()}')

    def get_solved_squares(self):
        return {key : val for key, val in b.empty_squares.items() if len(val)==1}

    def get_two_candidate_squares(self):
        return {key : val for key, val in b.empty_squares.items() if len(val)==2}
        
    def fill_in_solved_squares(self):
        single_candidates = self.get_solved_squares()
        for sq in single_candidates:
            self.set_square(sq, single_candidates[sq].pop())
            self.empty_squares.pop(sq)

    def summarize_board(self):
        return hash_count(list(map(len, self.empty_squares.values())), sort=True)

    def forward_solve(self):
        while len(self.empty_squares) > 0:
            self.fill_in_solved_squares()
            self.update_candidate_notes()
            if len(self.get_solved_squares()) == 0:
                break 

    def export_boardstring(self):
        # same format as original input file
        return f'{self.name}\n' + '\n'.join(list(map(lambda r: ''.join(r), self.rows)))

def flatten(matrix):
    return [item for row in matrix for item in row]

def hash_count(lst, sort=False):
    summary = {}
    for val in lst:
        summary[val] = 1 + summary.get(val, 0)
    if sort:
        return dict(sorted(summary.items()))
    else:
        return summary

def hash_count_bis(lst, sort=False):
    summary = {}
    for val in lst:
        if val[1] in summary:
            summary[val[1]].append(val)
        else:
            summary[val[1]] = [val]
    if sort:
        return dict(sorted(summary.items()))
    else:
        return summary

In [3]:
list('123456789')

['1', '2', '3', '4', '5', '6', '7', '8', '9']

In [4]:
def buildboard(chunk):
    b = {}
    lines = chunk.split('\n')
    b[lines[0]] = list(map(list, lines[1:10]))
    return b

In [8]:
#infile = 'sudoku.txt'
infile = 'p096_sudoku.txt'
boardchunks = []
boardlines = ''
with open(infile) as f:
    for line in f:        
        if line.startswith('Grid') and boardlines:
            boardchunks.append(boardlines)
            boardlines = line
        else:
            boardlines += line
boardchunks

['Grid 01\n003020600\n900305001\n001806400\n008102900\n700000008\n006708200\n002609500\n800203009\n005010300\n',
 'Grid 02\n200080300\n060070084\n030500209\n000105408\n000000000\n402706000\n301007040\n720040060\n004010003\n',
 'Grid 03\n000000907\n000420180\n000705026\n100904000\n050000040\n000507009\n920108000\n034059000\n507000000\n',
 'Grid 04\n030050040\n008010500\n460000012\n070502080\n000603000\n040109030\n250000098\n001020600\n080060020\n',
 'Grid 05\n020810740\n700003100\n090002805\n009040087\n400208003\n160030200\n302700060\n005600008\n076051090\n',
 'Grid 06\n100920000\n524010000\n000000070\n050008102\n000000000\n402700090\n060000000\n000030945\n000071006\n',
 'Grid 07\n043080250\n600000000\n000001094\n900004070\n000608000\n010200003\n820500000\n000000005\n034090710\n',
 'Grid 08\n480006902\n002008001\n900370060\n840010200\n003704100\n001060049\n020085007\n700900600\n609200018\n',
 'Grid 09\n000900002\n050123400\n030000160\n908000000\n070000090\n000000205\n091000050\n00743902

In [13]:
b = Board(boardchunks[1])
print(b)

Grid 02
[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '0', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]


In [15]:
b.forward_solve()
print(b)

Grid 02
[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]


In [11]:
# b.set_square((0, 1), '4')
# b.set_square((6, 4), '5')
b.empty_squares[(0,1)] = set(['4'])
b.empty_squares[(6,4)] = set(['5'])
print(b)


Grid 02
[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '0', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]


In [12]:
b.forward_solve()
print(b)

Grid 02
[['2', '4', '0', '9', '8', '1', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '4', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '5', '7', '0', '4', '2'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]


In [71]:
b.empty_squares[(0,8)] = set(['6'])
b.empty_squares[(4,3)] = set(['4'])
b.empty_squares[(4,7)] = set(['2'])
b.forward_solve()
print(b)

Grid 02
[['2', '4', '0', '9', '8', '1', '3', '0', '6'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '4', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '4', '0', '0', '0', '2', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '5', '7', '0', '4', '2'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]


In [73]:
b.empty_squares[(3,4)] = set(['2'])
b.empty_squares[(4,6)] = set(['6'])
b.empty_squares[(8,0)] = set(['6'])
b.forward_solve()
print(b)

Grid 02
[['2', '4', '0', '9', '8', '1', '3', '0', '6'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '4', '2', '0', '9'],
 ['9', '7', '6', '1', '2', '5', '4', '3', '8'],
 ['0', '0', '0', '4', '0', '0', '6', '2', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '5', '7', '0', '4', '2'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['6', '0', '4', '0', '1', '0', '0', '0', '3']]


In [75]:
b.empty_squares[(1,2)] = set(['9'])
b.empty_squares[(4,2)] = set(['3'])
b.empty_squares[(5,4)] = set(['3'])
b.empty_squares[(6,3)] = set(['6'])
b.forward_solve()
print(b)

Grid 02
[['2', '4', '0', '9', '8', '1', '3', '0', '6'],
 ['0', '6', '9', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '4', '2', '0', '9'],
 ['9', '7', '6', '1', '2', '5', '4', '3', '8'],
 ['0', '0', '3', '4', '9', '8', '6', '2', '0'],
 ['4', '0', '2', '7', '3', '6', '0', '0', '0'],
 ['3', '0', '1', '6', '5', '7', '0', '4', '2'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['6', '0', '4', '0', '1', '0', '0', '0', '3']]


In [77]:
b.empty_squares[(5,1)] = set(['8'])
b.forward_solve()
print(b)

Grid 02
[['2', '4', '5', '9', '8', '1', '3', '7', '6'],
 ['1', '6', '9', '2', '7', '3', '5', '8', '4'],
 ['8', '3', '7', '5', '6', '4', '2', '1', '9'],
 ['9', '7', '6', '1', '2', '5', '4', '3', '8'],
 ['5', '1', '3', '4', '9', '8', '6', '2', '7'],
 ['4', '8', '2', '7', '3', '6', '9', '5', '1'],
 ['3', '9', '1', '6', '5', '7', '8', '4', '2'],
 ['7', '2', '8', '3', '4', '9', '1', '6', '5'],
 ['6', '5', '4', '8', '1', '2', '7', '9', '3']]


In [78]:
for logline in b.solve_log:
    print(logline)

51 squares open. Candidates per square summary: {1: 1, 2: 12, 3: 16, 4: 13, 5: 7, 6: 2}
48 squares open. Candidates per square summary: {1: 2, 2: 14, 3: 12, 4: 12, 5: 6, 6: 2}
46 squares open. Candidates per square summary: {1: 1, 2: 17, 3: 12, 4: 10, 5: 4, 6: 2}
45 squares open. Candidates per square summary: {1: 1, 2: 18, 3: 11, 4: 9, 5: 4, 6: 2}
44 squares open. Candidates per square summary: {2: 18, 3: 11, 4: 10, 5: 3, 6: 2}
41 squares open. Candidates per square summary: {2: 19, 3: 12, 4: 6, 5: 3, 6: 1}
38 squares open. Candidates per square summary: {1: 1, 2: 19, 3: 10, 4: 6, 5: 2}
37 squares open. Candidates per square summary: {1: 1, 2: 20, 3: 11, 4: 5}
36 squares open. Candidates per square summary: {1: 1, 2: 20, 3: 12, 4: 3}
35 squares open. Candidates per square summary: {1: 1, 2: 19, 3: 13, 4: 2}
34 squares open. Candidates per square summary: {2: 19, 3: 13, 4: 2}
30 squares open. Candidates per square summary: {1: 1, 2: 17, 3: 10, 4: 2}
29 squares open. Candidates per squa

In [107]:
b.forward_solve()
print(b)

Grid 02
[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]


In [108]:
b.get_box_row(0)

[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '0', '2', '0', '9']]

In [109]:
b.get_box_col(0)

[['2', '0', '0', '0', '0', '4', '3', '7', '0'],
 ['0', '6', '3', '0', '0', '0', '0', '2', '0'],
 ['0', '0', '0', '0', '0', '2', '1', '0', '4']]

In [156]:
# box_col == 0 means the first column of boxes, i.e. the first 3 columns

digit_summary = hash_count(flatten(b.get_box_col(0)))
digit_summary.pop('0')
digit_summary
digits_filled_in_2_boxes_out_of_3 = {key : val for key, val in digit_summary.items() if val==2}
digits_filled_in_2_boxes_out_of_3

{'4': 2, '3': 2}

In [188]:
b.get_cols_containing_digit('4')

[0, 2, 4, 6, 7, 8]

In [16]:
b.get_box_col(0)

[['2', '0', '0', '0', '0', '4', '3', '7', '0'],
 ['0', '6', '3', '0', '0', '0', '0', '2', '0'],
 ['0', '0', '0', '0', '0', '2', '1', '0', '4']]

In [186]:
digit_summary_w_index = hash_count_bis(flatten(list(map(lambda x: list(enumerate(x)), b.get_box_col(0)))))
digit_summary_w_index.pop('0')
digit_summary_w_index
digits_filled_in_2_boxes_out_of_3_idx = {key : val for key, val in digit_summary_w_index.items() if len(val)==2}
digits_filled_in_2_boxes_out_of_3_idx

{'4': [(5, '4'), (8, '4')], '3': [(6, '3'), (2, '3')]}

In [18]:
b.get_cols_containing_digit('4')

[0, 2, 4, 6, 7, 8]

In [19]:
cols_in_box_col_0 = [0, 1, 2] 

In [20]:
column_needs_digit4 = set(cols_in_box_col_0).difference(set(b.get_cols_containing_digit('4'))).pop()
column_needs_digit4

1

In [157]:
b.get_rows_containing_digit('4')

[1, 3, 5, 6, 7, 8]

In [158]:
b.get_cols_containing_digit('4')

[0, 2, 4, 6, 7, 8]

In [160]:
b.get_boxes_containing_digit('4')

[2, 3, 5, 6, 7, 8]

In [21]:
boxes_in_box_col0  =[0, 3, 6]

In [23]:
box_neeeds_digit4 = set(boxes_in_box_col0).difference(set(b.get_boxes_containing_digit('4'))).pop()
box_neeeds_digit4

0

In [24]:
b.get_box(box_neeeds_digit4)

['2', '0', '0', '0', '6', '0', '0', '3', '0']

In [25]:
b.get_col(column_needs_digit4)

['0', '6', '3', '0', '0', '0', '0', '2', '0']

In [180]:
hash_count_bis(flatten(list(map(lambda x: list(enumerate(x)), b.get_box_col(0)))))

{'2': [(0, '2'), (7, '2'), (5, '2')],
 '0': [(1, '0'),
  (2, '0'),
  (3, '0'),
  (4, '0'),
  (8, '0'),
  (0, '0'),
  (3, '0'),
  (4, '0'),
  (5, '0'),
  (6, '0'),
  (8, '0'),
  (0, '0'),
  (1, '0'),
  (2, '0'),
  (3, '0'),
  (4, '0'),
  (7, '0')],
 '4': [(5, '4'), (8, '4')],
 '3': [(6, '3'), (2, '3')],
 '7': [(7, '7')],
 '6': [(1, '6')],
 '1': [(6, '1')]}

In [142]:
list(map(lambda x: '2' in x, b.rows))

[True, False, True, False, False, True, False, True, False]

In [147]:
[i for i, x in enumerate(map(lambda x: '2' in x, b.get_cols())) if x]


[0, 1, 2, 6]

In [143]:
list(filter(lambda x: '2' in x, b.rows))

[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '3', '0', '5', '0', '0', '2', '0', '9'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0']]

In [139]:
list(map(lambda x: x[0] if '2' in x[1] else None, enumerate(b.rows)))

[0, None, 2, None, None, 5, None, 7, None]

In [154]:
b.get_boxes_containing_digit('2')

[0, 2, 3, 6]

In [133]:
list(enumerate(b.rows))

[(0, ['2', '0', '0', '0', '8', '0', '3', '0', '0']),
 (1, ['0', '6', '0', '0', '7', '0', '0', '8', '4']),
 (2, ['0', '3', '0', '5', '0', '0', '2', '0', '9']),
 (3, ['0', '0', '0', '1', '0', '5', '4', '0', '8']),
 (4, ['0', '0', '0', '0', '0', '0', '0', '0', '0']),
 (5, ['4', '0', '2', '7', '0', '6', '0', '0', '0']),
 (6, ['3', '0', '1', '0', '0', '7', '0', '4', '0']),
 (7, ['7', '2', '0', '0', '4', '0', '0', '6', '0']),
 (8, ['0', '0', '4', '0', '1', '0', '0', '0', '3'])]

In [39]:
b.get_two_candidate_squares()

{(0, 3): {'4', '9'},
 (1, 2): {'5', '9'},
 (1, 6): {'1', '5'},
 (2, 0): {'1', '8'},
 (2, 2): {'7', '8'},
 (2, 5): {'1', '4'},
 (2, 7): {'1', '7'},
 (3, 0): {'6', '9'},
 (3, 1): {'7', '9'},
 (5, 4): {'3', '9'},
 (5, 8): {'1', '5'},
 (6, 8): {'2', '5'},
 (7, 8): {'1', '5'}}

In [44]:
b.set_square((0,3), 9)

In [45]:
b.update_candidate_notes()

In [40]:
b.empty_squares.pop((0,3))

{'4', '9'}

In [ ]:
print(''.join(b.rows[0]))

In [ ]:
# print(f'{b.name}\n' + '\n'.join(list(map(lambda r: ''.join(r), b.rows))))
print(b.export_boardstring())

In [ ]:
#for sq in b.empty_squares:
#    b.calc_candidates(sq)
#b.update_candidate_notes()
hash_count(list(map(len, b.empty_squares.values())), sort=True)

In [ ]:
print(len(b.empty_squares))

In [ ]:
b.fill_in_solved_squares()

In [ ]:
print(b)

In [ ]:
b.empty_squares

In [ ]:
res = {key : val for key, val in b.empty_squares.items()
                   if len(val)==1}
res

In [22]:
for chunk in boardchunks:
    b = Board(chunk)
    b.forward_solve()
    print(b.name)
    for logline in b.solve_log:
        print(logline)
    

Grid 01
49 squares open. Candidates per square summary: {1: 3, 2: 14, 3: 10, 4: 13, 5: 8, 6: 1}
46 squares open. Candidates per square summary: {1: 5, 2: 13, 3: 12, 4: 11, 5: 5}
41 squares open. Candidates per square summary: {1: 10, 2: 7, 3: 15, 4: 5, 5: 4}
31 squares open. Candidates per square summary: {1: 5, 2: 15, 3: 4, 4: 6, 5: 1}
26 squares open. Candidates per square summary: {1: 3, 2: 14, 3: 5, 4: 3, 5: 1}
23 squares open. Candidates per square summary: {1: 3, 2: 12, 3: 6, 4: 2}
20 squares open. Candidates per square summary: {1: 3, 2: 11, 3: 6}
17 squares open. Candidates per square summary: {1: 5, 2: 10, 3: 2}
12 squares open. Candidates per square summary: {1: 10, 2: 2}
2 squares open. Candidates per square summary: {1: 2}
0 squares open. Candidates per square summary: {}
Grid 02
51 squares open. Candidates per square summary: {1: 1, 2: 12, 3: 16, 4: 13, 5: 7, 6: 2}
50 squares open. Candidates per square summary: {2: 13, 3: 16, 4: 12, 5: 7, 6: 2}
Grid 03
53 squares open. Ca

In [ ]:
boards = {}

key = 'Grid 01'
b = []

boardlines = 'Grid 01\n'
with open(infile) as f:
    for line in f:
        linestr = line.rstrip()
#        print(linestr)
        
        if linestr.startswith('Grid'):
            boards[key] = b
            key = linestr
            b = []
        else:
            linelist = list(linestr)
            b.append(linelist)
#            print(linelist)
boards

In [ ]:
Box 0:
[self.rows[0][0:3], self.rows[1][0:3], self.rows[2][0:3] ]
Box 1:
[self.rows[0][3:6], self.rows[1][3:6], self.rows[2][3:6] ]

Box 2:
[self.rows[0][6:9], self.rows[1][6:9], self.rows[2][6:9] ]

Box 3:
[self.rows[3][0:3], self.rows[4][0:3], self.rows[5][0:3] ]

Box 6:
[self.rows[6][0:3], self.rows[7][0:3], self.rows[8][0:3] ]
Box 7:
[self.rows[6][3:6], self.rows[7][3:6], self.rows[8][3:6] ]


In [ ]:
for box in [0, 1, 2, 3, 4, 5, 6, 7, 8]:
#    print(f'box: {box} b_row:{box//3} b_col:{box%3}')
    print(f'({box//3}, {box%3}) : {box},')

In [81]:
b.rows[0:3]

[['2', '4', '5', '9', '8', '1', '3', '7', '6'],
 ['1', '6', '9', '2', '7', '3', '5', '8', '4'],
 ['8', '3', '7', '5', '6', '4', '2', '1', '9']]

In [82]:
b.rows

[['2', '4', '5', '9', '8', '1', '3', '7', '6'],
 ['1', '6', '9', '2', '7', '3', '5', '8', '4'],
 ['8', '3', '7', '5', '6', '4', '2', '1', '9'],
 ['9', '7', '6', '1', '2', '5', '4', '3', '8'],
 ['5', '1', '3', '4', '9', '8', '6', '2', '7'],
 ['4', '8', '2', '7', '3', '6', '9', '5', '1'],
 ['3', '9', '1', '6', '5', '7', '8', '4', '2'],
 ['7', '2', '8', '3', '4', '9', '1', '6', '5'],
 ['6', '5', '4', '8', '1', '2', '7', '9', '3']]

In [83]:
b.get_box(3)

['9', '7', '6', '5', '1', '3', '4', '8', '2']

In [121]:
list(map(lambda x: b.get_col(x), range(0,9)))

[['2', '0', '0', '0', '0', '4', '3', '7', '0'],
 ['0', '6', '3', '0', '0', '0', '0', '2', '0'],
 ['0', '0', '0', '0', '0', '2', '1', '0', '4'],
 ['0', '0', '5', '1', '0', '7', '0', '0', '0'],
 ['8', '7', '6', '0', '0', '0', '0', '4', '1'],
 ['0', '0', '0', '5', '0', '6', '7', '0', '0'],
 ['3', '0', '2', '4', '0', '0', '0', '0', '0'],
 ['0', '8', '0', '0', '0', '0', '4', '6', '0'],
 ['0', '4', '9', '8', '0', '0', '0', '0', '3']]

In [128]:
b.rows

[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '0', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]

In [129]:
b.get_rows()

[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '0', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]

In [130]:
b.get_cols()

[['2', '0', '0', '0', '0', '4', '3', '7', '0'],
 ['0', '6', '3', '0', '0', '0', '0', '2', '0'],
 ['0', '0', '0', '0', '0', '2', '1', '0', '4'],
 ['0', '0', '5', '1', '0', '7', '0', '0', '0'],
 ['8', '7', '0', '0', '0', '0', '0', '4', '1'],
 ['0', '0', '0', '5', '0', '6', '7', '0', '0'],
 ['3', '0', '2', '4', '0', '0', '0', '0', '0'],
 ['0', '8', '0', '0', '0', '0', '4', '6', '0'],
 ['0', '4', '9', '8', '0', '0', '0', '0', '3']]

In [131]:
b.get_boxes()

[['2', '0', '0', '0', '6', '0', '0', '3', '0'],
 ['0', '8', '0', '0', '7', '0', '5', '0', '0'],
 ['3', '0', '0', '0', '8', '4', '2', '0', '9'],
 ['0', '0', '0', '0', '0', '0', '4', '0', '2'],
 ['1', '0', '5', '0', '0', '0', '7', '0', '6'],
 ['4', '0', '8', '0', '0', '0', '0', '0', '0'],
 ['3', '0', '1', '7', '2', '0', '0', '0', '4'],
 ['0', '0', '7', '0', '4', '0', '0', '1', '0'],
 ['0', '4', '0', '0', '6', '0', '0', '0', '3']]